参考资料：<br>
白话word2vec：https://zhuanlan.zhihu.com/p/81032021 <br>
什么是词向量？https://blog.csdn.net/mawenqi0729/article/details/80698350 <br>
官方参数解读：https://tedboy.github.io/nlps/generated/generated/gensim.models.Word2Vec.html#gensim.models.Word2Vec <br>
为什么PCA：https://zhuanlan.zhihu.com/p/37810506 <br>
https://blog.csdn.net/HLBoy_happy/article/details/77146012 <br>
参数解读博客版：https://blog.csdn.net/xiaoQL520/article/details/102509477 <br>
负采样：https://zhuanlan.zhihu.com/p/144146838 <br>
更多资料：https://zhuanlan.zhihu.com/p/26306795 <br>
https://mp.weixin.qq.com/s/j8JPMZSPoVT_hQswX5QVxA

In [4]:
import jieba
import re
import numpy as np
from sklearn.decomposition import PCA
import gensim
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import matplotlib

## 分词

In [5]:
f = open("sanguo.txt", 'r',encoding='utf-8') #读入文本
lines = []
for line in f: #分别对每段分词
    temp = jieba.lcut(line)  #结巴分词 精确模式
    words = []
    for i in temp:
        #过滤掉所有的标点符号
        i = re.sub("[\s+\.\!\/_,$%^*(+\"\'””《》]+|[+——！，。？、~@#￥%……&*（）：；‘]+", "", i)
        if len(i) > 0:
            words.append(i)
    if len(words) > 0:
        lines.append(words)
print(lines[0:5])#预览前5行分词结果

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\BGM\AppData\Local\Temp\jieba.cache
Loading model cost 0.637 seconds.
Prefix dict has been built successfully.


[['三国演义', '上卷'], ['罗贯中'], ['滚滚', '长江', '东', '逝水', '浪花', '淘尽', '英雄', '是非成败', '转头', '空', '青山', '依旧', '在', '几度', '夕阳红'], ['白发', '渔樵', '江渚上', '惯看', '秋月春风', '一壶', '浊酒', '喜相逢', '古今', '多少', '事', '都', '付笑谈', '中'], ['--', '调寄', '临江仙']]


## 模型训练 

In [6]:
# 调用Word2Vec训练
# 参数：size: 词向量维度；window: 上下文的宽度，min_count为考虑计算的单词的最低词频阈值
model = Word2Vec(lines,vector_size = 20, window = 2 , min_count = 3, epochs=7, negative=10,sg=1)
print("孔明的词向量：\n",model.wv.get_vector('孔明'))
print("\n和孔明相关性最高的前20个词语：")
model.wv.most_similar('孔明', topn = 20)# 与孔明最相关的前20个词语

孔明的词向量：
 [-0.15494478  0.17697874  0.1343293  -0.2487949   0.1767206  -0.4730292
  0.05967851  1.1771556   0.33726934  0.23582324 -0.32211086 -0.18821828
  0.12791814 -0.9263279   1.0261534   0.9537278   0.39280412  0.04630988
 -1.1328075  -0.58372957]

和孔明相关性最高的前20个词语：


[('先主', 0.9126538634300232),
 ('玄德', 0.9124650955200195),
 ('周瑜', 0.9073388576507568),
 ('关公', 0.8809785842895508),
 ('维', 0.8788902163505554),
 ('门吏', 0.8788679838180542),
 ('袁术', 0.8706308007240295),
 ('司马昭', 0.8679661750793457),
 ('成都', 0.8672823309898376),
 ('钟会', 0.8615831732749939),
 ('使人', 0.8588334918022156),
 ('孙权', 0.8587878942489624),
 ('魏主', 0.851751983165741),
 ('懿', 0.8510929346084595),
 ('陆逊', 0.8499301075935364),
 ('璋', 0.8468621373176575),
 ('孟获', 0.8450707793235779),
 ('孔明入', 0.8440046310424805),
 ('庞统', 0.8433902859687805),
 ('孙夫人', 0.842359721660614)]

## 可视化 

In [7]:
# 将词向量投影到二维空间
rawWordVec = []
word2ind = {}
for i, w in enumerate(model.wv.index_to_key): #index_to_key 序号,词语
    rawWordVec.append(model.wv[w]) #词向量
    word2ind[w] = i #{词语:序号}
rawWordVec = np.array(rawWordVec)
X_reduced = PCA(n_components=2).fit_transform(rawWordVec) 

In [8]:
rawWordVec #降维之前20维

array([[ 0.21720165,  0.49328578, -0.48997474, ..., -0.37047064,
        -1.6601005 , -1.4215053 ],
       [-0.13916075,  1.0265715 ,  0.29569283, ..., -0.57129145,
        -1.0583715 , -1.0433223 ],
       [ 0.31796825,  0.98545694, -0.18428032, ..., -0.46424124,
        -1.1205398 , -0.72955537],
       ...,
       [-0.02093252,  0.260414  , -0.20132133, ..., -0.4450067 ,
        -0.51022357, -0.23723757],
       [-0.15698686,  0.2876415 , -0.2243969 , ..., -0.46064842,
        -0.52135944, -0.2128545 ],
       [-0.1817357 ,  0.31027526, -0.27303386, ..., -0.3751892 ,
        -0.5678903 , -0.1117091 ]], dtype=float32)

In [9]:
X_reduced #降维之后2维

array([[-1.4918272 ,  0.07349118],
       [-0.8817787 ,  0.11658159],
       [-1.1799757 ,  0.01956591],
       ...,
       [-0.02821559, -0.17496787],
       [ 0.14210726, -0.21936099],
       [ 0.21188387, -0.24971846]], dtype=float32)

In [10]:
# 绘制星空图
# 绘制所有单词向量的二维空间投影
fig = plt.figure(figsize = (15, 10))
ax = fig.gca()
ax.set_facecolor('white')
ax.plot(X_reduced[:, 0], X_reduced[:, 1], '.', markersize = 1, alpha = 0.3, color = 'black')


# 绘制几个特殊单词的向量
words = ['孙权', '刘备', '曹操', '周瑜', '诸葛亮', '司马懿','汉献帝']

# 设置中文字体 否则乱码
zhfont1 = matplotlib.font_manager.FontProperties(fname='./华文仿宋.ttf', size=16)
for w in words:
    if w in word2ind:
        ind = word2ind[w]
        xy = X_reduced[ind]
        plt.plot(xy[0], xy[1], '.', alpha =1, color = 'orange',markersize=10)
        plt.text(xy[0], xy[1], w, fontproperties = zhfont1, alpha = 1, color = 'red')

Error in callback <function _draw_all_if_interactive at 0x00000289D32AD300> (for post_execute):


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Github\\AILab\\nlp\\word2vec\\华文仿宋.ttf'

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Github\\AILab\\nlp\\word2vec\\华文仿宋.ttf'

<Figure size 1500x1000 with 1 Axes>

## 类比关系实验

In [11]:
# 玄德－孔明＝？－曹操
words = model.wv.most_similar(positive=['玄德', '曹操'], negative=['孔明'])
words

[('司马懿', 0.9146393537521362),
 ('袁绍', 0.8516148328781128),
 ('曹真', 0.8143850564956665),
 ('吴侯', 0.8098608255386353),
 ('周瑜', 0.8090276718139648),
 ('魏主', 0.8031066656112671),
 ('吕布', 0.7986387014389038),
 ('公孙瓒', 0.7970840334892273),
 ('东吴', 0.7944876551628113),
 ('蔡瑁', 0.7916224598884583)]

In [12]:
# 曹操－魏＝？－蜀
words = model.wv.most_similar(positive=['曹操', '蜀'], negative=['魏'])
words

[('袁绍', 0.7129155397415161),
 ('马超', 0.6938760280609131),
 ('吕布', 0.6416083574295044),
 ('刘备', 0.6267032623291016),
 ('五路', 0.6258271932601929),
 ('吴兵', 0.6210716962814331),
 ('周瑜', 0.6208724975585938),
 ('早', 0.619978129863739),
 ('邓艾', 0.6102068424224854),
 ('司马懿', 0.6093023419380188)]

In [15]:
words = model.wv.most_similar(positive=['关羽', '曹操'], negative=['赵云'])
words

[('中原', 0.890377938747406),
 ('以来', 0.8663949370384216),
 ('使君', 0.8639323711395264),
 ('皇帝', 0.853752851486206),
 ('实为', 0.8516172170639038),
 ('备', 0.841305136680603),
 ('幸甚', 0.8405286073684692),
 ('恢复', 0.8394755125045776),
 ('天子', 0.8383244276046753),
 ('之重', 0.834486722946167)]